**Laboratorio 2**

**Clasificador de canciones de Spotify**

Grupo 30:

• Aniñir Lionel

• Leal Patricia Guillermina

• Weinzettel Eduardo

In [1]:
# Importar bibliotecas necesarias
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Cargar el conjunto de datos desde el archivo CSV
data = pd.read_csv("/content/drive/MyDrive/BIG DATA/LABORATORIO II/Canciones_Spotify.csv")

# Mostrar las primeras filas del DataFrame para entender la estructura de los datos
data

,Unnamed: 0,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,target,song_title,artist
0,0,0.01020,0.833,204600,0.434,0.021900,2,0.1650,-8.795,1,0.4310,150.062,4.0,0.286,1,Mask Off,Future
1,1,0.19900,0.743,326933,0.359,0.006110,1,0.1370,-10.401,1,0.0794,160.083,4.0,0.588,1,Redbone,Childish Gambino
2,2,0.03440,0.838,185707,0.412,0.000234,2,0.1590,-7.148,1,0.2890,75.044,4.0,0.173,1,Xanny Family,Future
3,3,0.60400,0.494,199413,0.338,0.510000,5,0.0922,-15.236,1,0.0261,86.468,4.0,0.230,1,Master Of None,Beach House
4,4,0.18000,0.678,392893,0.561,0.512000,5,0.4390,-11.648,0,0.0694,174.004,4.0,0.904,1,Parallel Lines,Junior Boys
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012,2012,0.00106,0.584,274404,0.932,0.002690,1,0.1290,-3.501,1,0.3330,74.976,4.0,0.211,0,Like A Bitch - Kill The Noise Remix,Kill The Noise
2013,2013,0.08770,0.894,182182,0.892,0.001670,1,0.0528,-2.663,1,0.1310,110.041,4.0,0.867,0,Candy,Dillon Francis
2014,2014,0.00857,0.637,207200,0.935,0.003990,0,0.2140,-2.467,1,0.1070,150.082,4.0,0.470,0,Habit - Dack Janiels & Wenzday Remix,Rain Man
2015,2015,0.00164,0.557,185600,0.992,0.677000,1,0.0913,-2.735,1,0.1330,150.011,4.0,0.623,0,First Contact,Twin Moons


In [5]:
from sklearn.preprocessing import StandardScaler, LabelEncoder #Importar biblioteca necesaria

# Eliminar columnas innecesarias y manejar variables categóricas
data = data.drop(columns=["Unnamed: 0", "song_title", "artist"])
label_encoder = LabelEncoder()
data["target"] = label_encoder.fit_transform(data["target"])
# Usamos LabelEncoder para convertir la variable objetivo ('target') en valores numéricos.
# Esto es necesario para que los modelos de Machine Learning puedan trabajar con la variable objetivo.

# Separar características (X) y etiquetas (y)
X = data.drop(columns=["target"]) # X contiene las características, que son todas las columnas excepto 'target'.
y = data["target"] # y contiene la variable objetivo, que es la columna 'target'.

# Escalar características numéricas
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
# Estandarizamos las características numéricas para que tengan una media de 0 y una desviación estándar de 1.
# Esto es importante para modelos como SVM y KNN, que son sensibles a la escala de las características.

# Dividir los datos en conjuntos de entrenamiento y prueba
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
# Dividimos los datos en un conjunto de entrenamiento (80%) y un conjunto de prueba (20%).
# El parámetro random_state=42 asegura que la división sea reproducible, utilizando la misma semilla aleatoria cada vez que se ejecuta el código.

# En este punto, hemos preparado los datos para entrenar y evaluar los modelos de Machine Learning.

# KNN (K-Nearest Neighbors)

El algoritmo KNN es un método de aprendizaje supervisado que se utiliza para clasificación y regresión. Funciona buscando las k observaciones más similares en el conjunto de entrenamiento para una nueva observación y realiza una predicción basada en sus vecinos más cercanos.

Primero, entrenamos un modelo KNN con los datos de entrenamiento y luego evaluamos su rendimiento utilizando diferentes métricas.

Inicialización del clasificador KNN: Creamos un clasificador KNN con 5 vecinos más cercanos. Este número de vecinos es un valor arbitrario y puede ser ajustado según sea necesario.
Entrenamiento del modelo: El modelo se entrena utilizando los datos de entrenamiento (X_train y y_train). Durante el entrenamiento, el modelo aprende los patrones presentes en los datos.
Predicciones: Utilizamos el modelo entrenado para hacer predicciones sobre el conjunto de prueba (X_test). Estas predicciones se almacenan en y_pred_knn.
Evaluación del rendimiento: Calculamos la matriz de confusión y el informe de clasificación para evaluar el rendimiento del modelo KNN en el conjunto de prueba.

In [6]:
from sklearn.metrics import confusion_matrix, classification_report

# Inicializar el clasificador KNN con un valor de k arbitrario (por ejemplo, k=5)
knn_classifier = KNeighborsClassifier(n_neighbors=5)
# Usamos 5 vecinos más cercanos para hacer las predicciones.

# Entrenar el modelo KNN con los datos de entrenamiento
knn_classifier.fit(X_train, y_train)
# El modelo se ajusta a los datos de entrenamiento, aprendiendo los patrones presentes en los datos.

# Realizar predicciones en el conjunto de prueba
y_pred_knn = knn_classifier.predict(X_test)
# Usamos el modelo entrenado para hacer predicciones sobre el conjunto de prueba.

# Evaluar el rendimiento del modelo KNN
conf_matrix_knn = confusion_matrix(y_test, y_pred_knn)
# Creamos una matriz de confusión para evaluar el rendimiento del modelo.
# La matriz de confusión muestra la cantidad de verdaderos positivos, falsos positivos, verdaderos negativos y falsos negativos.

class_report_knn = classification_report(y_test, y_pred_knn)
# Creamos un informe de clasificación que incluye precision, recall y F1-score para evaluar el rendimiento del modelo.

# Imprimir la matriz de confusión y el informe de clasificación del modelo KNN
print("Matriz de Confusión (KNN):")
print(conf_matrix_knn)
print("\nInforme de Clasificación (KNN):")
print(class_report_knn)

Matriz de Confusión (KNN):
[[158  48]
 [ 65 133]]

Informe de Clasificación (KNN):
              precision    recall  f1-score   support

           0       0.71      0.77      0.74       206
           1       0.73      0.67      0.70       198

    accuracy                           0.72       404
   macro avg       0.72      0.72      0.72       404
weighted avg       0.72      0.72      0.72       404



# SVM (Support Vector Machines)

Modelo Support Vector Machines (SVM)
SVM son un conjunto versátil de métodos de aprendizaje supervisado que se utilizan tanto para clasificación como para regresión. SVM funciona encontrando el hiperplano que mejor separa las clases en un espacio multidimensional. Podemos usar distintos kernel para SVM, como lineal, polinómico o radial (RBF). En este caso, usamos el kernel radial.

Inicialización del clasificador SVM: Creamos un clasificador SVM con kernel radial. Utilizamos el kernel radial (RBF) porque es adecuado para datos que no tienen una frontera de decisión linealmente separable.
Entrenamiento del modelo: El modelo se entrena utilizando los datos de entrenamiento (X_train y y_train). Durante el entrenamiento, el modelo encuentra el hiperplano que mejor separa las clases en el espacio multidimensional.
Predicciones: Utilizamos el modelo entrenado para hacer predicciones sobre el conjunto de prueba (X_test). Estas predicciones se almacenan en y_pred_svm.
Evaluación del rendimiento: Calculamos la matriz de confusión y el informe de clasificación para evaluar el rendimiento del modelo SVM en el conjunto de prueba.

In [7]:
# Inicializar el clasificador SVM con kernel radial
svm_classifier = SVC(kernel='rbf', random_state=42)
# Usamos el kernel radial ('rbf') para SVM.

# Entrenar el modelo SVM con los datos de entrenamiento
svm_classifier.fit(X_train, y_train)
# El modelo se ajusta a los datos de entrenamiento, aprendiendo los patrones presentes en los datos.

# Realizar predicciones en el conjunto de prueba
y_pred_svm = svm_classifier.predict(X_test)
# Usamos el modelo entrenado para hacer predicciones sobre el conjunto de prueba.

# Evaluar el rendimiento del modelo SVM
conf_matrix_svm = confusion_matrix(y_test, y_pred_svm)
# Creamos una matriz de confusión para evaluar el rendimiento del modelo SVM.

class_report_svm = classification_report(y_test, y_pred_svm)
# Creamos un informe de clasificación que incluye precision, recall y F1-score para evaluar el rendimiento del modelo SVM.

# Imprimir la matriz de confusión y el informe de clasificación del modelo SVM
print("Matriz de Confusión (SVM):")
print(conf_matrix_svm)
print("\nInforme de Clasificación (SVM):")
print(class_report_svm)

Matriz de Confusión (SVM):
[[164  42]
 [ 64 134]]

Informe de Clasificación (SVM):
              precision    recall  f1-score   support

           0       0.72      0.80      0.76       206
           1       0.76      0.68      0.72       198

    accuracy                           0.74       404
   macro avg       0.74      0.74      0.74       404
weighted avg       0.74      0.74      0.74       404



# Árbol de decisión

Modelo Árbol de Decisión
Los árboles de decisión son algoritmos de aprendizaje supervisado utilizados para clasificación y regresión. El modelo divide el conjunto de datos en subconjuntos más pequeños basados en ciertas características, haciendo preguntas sucesivas en cada nodo del árbol. Esto permite tomar decisiones basadas en las respuestas a esas preguntas.

Inicialización del clasificador de Árbol de Decisión: Creamos un clasificador de Árbol de Decisión sin especificar hiperparámetros adicionales. En este caso, utilizamos los valores predeterminados del algoritmo.
Entrenamiento del modelo: El modelo se entrena utilizando los datos de entrenamiento (X_train y y_train). Durante el entrenamiento, el modelo encuentra los patrones presentes en los datos utilizando preguntas basadas en las características.
Predicciones: Utilizamos el modelo entrenado para hacer predicciones sobre el conjunto de prueba (X_test). Estas predicciones se almacenan en y_pred_decision_tree.
Evaluación del rendimiento: Calculamos la matriz de confusión y el informe de clasificación para evaluar el rendimiento del modelo de Árbol de Decisión en el conjunto de prueba.

In [8]:
# Inicializar el clasificador de Árbol de Decisión
decision_tree_classifier = DecisionTreeClassifier(random_state=42)
# No especificamos ningún hiperparámetro, utilizaremos los valores predeterminados.

# Entrenar el modelo de Árbol de Decisión con los datos de entrenamiento
decision_tree_classifier.fit(X_train, y_train)
# El modelo se ajusta a los datos de entrenamiento, aprendiendo los patrones presentes en los datos.

# Realizar predicciones en el conjunto de prueba
y_pred_decision_tree = decision_tree_classifier.predict(X_test)
# Usamos el modelo entrenado para hacer predicciones sobre el conjunto de prueba.

# Evaluar el rendimiento del modelo de Árbol de Decisión
conf_matrix_decision_tree = confusion_matrix(y_test, y_pred_decision_tree)
# Creamos una matriz de confusión para evaluar el rendimiento del modelo de Árbol de Decisión.

class_report_decision_tree = classification_report(y_test, y_pred_decision_tree)
# Creamos un informe de clasificación que incluye precision, recall y F1-score para evaluar el rendimiento del modelo de Árbol de Decisión.

# Imprimir la matriz de confusión y el informe de clasificación del modelo de Árbol de Decisión
print("Matriz de Confusión (Árbol de Decisión):")
print(conf_matrix_decision_tree)
print("\nInforme de Clasificación (Árbol de Decisión):")
print(class_report_decision_tree)

Matriz de Confusión (Árbol de Decisión):
[[148  58]
 [ 62 136]]

Informe de Clasificación (Árbol de Decisión):
              precision    recall  f1-score   support

           0       0.70      0.72      0.71       206
           1       0.70      0.69      0.69       198

    accuracy                           0.70       404
   macro avg       0.70      0.70      0.70       404
weighted avg       0.70      0.70      0.70       404



# Bayes (Naive Bayes)

Modelo Naive Bayes
Naive Bayes es un algoritmo de aprendizaje supervisado basado en el teorema de Bayes. A pesar de su simplicidad, a menudo se utiliza para problemas de clasificación de texto y tiene buenos resultados en diversas aplicaciones.
Inicialización del clasificador Naive Bayes: Creamos un clasificador Naive Bayes utilizando el modelo Gaussian Naive Bayes, adecuado para características continuas como las nuestras.
Entrenamiento del modelo: El modelo se entrena utilizando los datos de entrenamiento (X_train y y_train). Durante el entrenamiento, el modelo aprende los patrones presentes en los datos utilizando el teorema de Bayes.
Predicciones: Utilizamos el modelo entrenado para hacer predicciones sobre el conjunto de prueba (X_test). Estas predicciones se almacenan en y_pred_naive_bayes.
Evaluación del rendimiento: Calculamos la matriz de confusión y el informe de clasificación para evaluar el rendimiento del modelo Naive Bayes en el conjunto de prueba.

In [9]:
from sklearn.naive_bayes import GaussianNB

# Inicializar el clasificador Naive Bayes
naive_bayes_classifier = GaussianNB()
# Utilizamos el modelo Gaussian Naive Bayes, que es adecuado para características continuas como las nuestras.

# Entrenar el modelo Naive Bayes con los datos de entrenamiento
naive_bayes_classifier.fit(X_train, y_train)
# El modelo se ajusta a los datos de entrenamiento, aprendiendo los patrones presentes en los datos.

# Realizar predicciones en el conjunto de prueba
y_pred_naive_bayes = naive_bayes_classifier.predict(X_test)
# Usamos el modelo entrenado para hacer predicciones sobre el conjunto de prueba.

# Evaluar el rendimiento del modelo Naive Bayes
conf_matrix_naive_bayes = confusion_matrix(y_test, y_pred_naive_bayes)
# Creamos una matriz de confusión para evaluar el rendimiento del modelo Naive Bayes.

class_report_naive_bayes = classification_report(y_test, y_pred_naive_bayes)
# Creamos un informe de clasificación que incluye precision, recall y F1-score para evaluar el rendimiento del modelo Naive Bayes.

# Imprimir la matriz de confusión y el informe de clasificación del modelo Naive Bayes
print("Matriz de Confusión (Naive Bayes):")
print(conf_matrix_naive_bayes)
print("\nInforme de Clasificación (Naive Bayes):")
print(class_report_naive_bayes)

Matriz de Confusión (Naive Bayes):
[[116  90]
 [ 58 140]]

Informe de Clasificación (Naive Bayes):
              precision    recall  f1-score   support

           0       0.67      0.56      0.61       206
           1       0.61      0.71      0.65       198

    accuracy                           0.63       404
   macro avg       0.64      0.64      0.63       404
weighted avg       0.64      0.63      0.63       404




# Validación Cruzada k-fold

Validación Cruzada k-fold
La validación cruzada k-fold divide el conjunto de datos en k partes iguales, llamadas pliegues. El modelo se entrena k veces, utilizando k-1 pliegues para entrenamiento y 1 pliegue para validación en cada iteración. Luego se promedian los resultados de las k iteraciones para obtener una estimación más precisa del rendimiento del modelo.
Utilizamos la función cross_val_score para realizar la validación cruzada k-fold para cada modelo (KNN, SVM, Naive Bayes) utilizando 5 pliegues (cv=5).
El parámetro scoring='accuracy' indica que estamos interesados en medir la precisión del modelo durante la validación cruzada.
Imprimimos los puntajes de validación cruzada obtenidos para cada modelo y calculamos la precisión promedio para evaluar el rendimiento general de cada modelo.

In [10]:
from sklearn.model_selection import cross_val_score

# Realizar validación cruzada k-fold para el modelo KNN
knn_scores = cross_val_score(knn_classifier, X_scaled, y, cv=5, scoring='accuracy')
print("Puntajes de Validación Cruzada (KNN):", knn_scores)
print("Precisión Promedio (KNN):", knn_scores.mean())

# Realizar validación cruzada k-fold para el modelo SVM
svm_scores = cross_val_score(svm_classifier, X_scaled, y, cv=5, scoring='accuracy')
print("\nPuntajes de Validación Cruzada (SVM):", svm_scores)
print("Precisión Promedio (SVM):", svm_scores.mean())

# Realizar validación cruzada k-fold para el modelo Naive Bayes
naive_bayes_scores = cross_val_score(naive_bayes_classifier, X_scaled, y, cv=5, scoring='accuracy')
print("\nPuntajes de Validación Cruzada (Naive Bayes):", naive_bayes_scores)
print("Precisión Promedio (Naive Bayes):", naive_bayes_scores.mean())

Puntajes de Validación Cruzada (KNN): [0.7029703  0.65841584 0.5483871  0.66997519 0.67990074]
Precisión Promedio (KNN): 0.6519298331818293

Puntajes de Validación Cruzada (SVM): [0.75742574 0.70792079 0.5483871  0.69230769 0.69727047]
Precisión Promedio (SVM): 0.6806623590398743

Puntajes de Validación Cruzada (Naive Bayes): [0.5        0.62376238 0.4764268  0.58312655 0.52109181]
Precisión Promedio (Naive Bayes): 0.5408815075055893


Optimización de Hiperparámetros.

Grid Search.

Grid Search es una técnica que evalúa exhaustivamente una cuadrícula predefinida de hiperparámetros para encontrar la combinación óptima. En este caso, vamos a usar Grid Search para los modelos KNN y SVM.
Definimos una cuadrícula de hiperparámetros para KNN, incluyendo el número de vecinos (n_neighbors), los pesos (weights) y la métrica de distancia (metric).
Configuramos Grid Search utilizando GridSearchCV. La validación cruzada con 5 pliegues (cv=5) se utiliza para evaluar cada combinación de hiperparámetros.
Realizamos Grid Search para encontrar los mejores hiperparámetros para el modelo KNN.

In [11]:
# Definir la cuadrícula de hiperparámetros para KNN
knn_param_grid = {
    'n_neighbors': [3, 5, 7, 9],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

# Inicializar el clasificador KNN
knn_classifier = KNeighborsClassifier()

# Configurar Grid Search para KNN
grid_search_knn = GridSearchCV(knn_classifier, knn_param_grid, cv=5, scoring='accuracy')

# Realizar Grid Search para KNN
grid_search_knn.fit(X_scaled, y)

# Obtener los mejores hiperparámetros para KNN
best_params_knn = grid_search_knn.best_params_
print("Mejores Hiperparámetros para KNN:", best_params_knn)

Mejores Hiperparámetros para KNN: {'metric': 'manhattan', 'n_neighbors': 7, 'weights': 'distance'}


Optimización de Hiperparámetros (Continuación)
Grid Search para SVM

Definimos una cuadrícula de hiperparámetros para SVM, incluyendo el parámetro de regularización (C), el tipo de kernel (kernel) y el coeficiente del kernel (gamma).
Configuramos Grid Search utilizando GridSearchCV. La validación cruzada con 5 pliegues (cv=5) se utiliza para evaluar cada combinación de hiperparámetros.
Realizamos Grid Search para encontrar los mejores hiperparámetros para el modelo SVM.

In [12]:
# Definir la cuadrícula de hiperparámetros para SVM
svm_param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']
}

# Inicializar el clasificador SVM
svm_classifier = SVC(random_state=42)

# Configurar Grid Search para SVM
grid_search_svm = GridSearchCV(svm_classifier, svm_param_grid, cv=5, scoring='accuracy')

# Realizar Grid Search para SVM
grid_search_svm.fit(X_scaled, y)

# Obtener los mejores hiperparámetros para SVM
best_params_svm = grid_search_svm.best_params_
print("Mejores Hiperparámetros para SVM:", best_params_svm)

Mejores Hiperparámetros para SVM: {'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}


Randomized Search para Naive Bayes

Definimos una distribución de hiperparámetros para Naive Bayes, incluyendo el parámetro de suavizado (var_smoothing). Utilizamos una escala logarítmica para explorar diferentes órdenes de magnitud.
Configuramos Randomized Search utilizando RandomizedSearchCV. La validación cruzada con 5 pliegues (cv=5) se utiliza para evaluar cada combinación de hiperparámetros. Realizamos 100 iteraciones (n_iter=100) para explorar diversas combinaciones de hiperparámetros de manera eficiente.
Realizamos Randomized Search para encontrar los mejores hiperparámetros para el modelo Naive Bayes.

In [13]:
import numpy as np

# Definir la distribución de hiperparámetros para Naive Bayes
naive_bayes_param_dist = {
    'var_smoothing': np.logspace(0, -9, num=100)
}

# Inicializar el clasificador Naive Bayes
naive_bayes_classifier = GaussianNB()

# Configurar Randomized Search para Naive Bayes
randomized_search_naive_bayes = RandomizedSearchCV(naive_bayes_classifier, param_distributions=naive_bayes_param_dist,
                                                   n_iter=100, cv=5, scoring='accuracy', random_state=42)

# Realizar Randomized Search para Naive Bayes
randomized_search_naive_bayes.fit(X_scaled, y)

# Obtener los mejores hiperparámetros para Naive Bayes
best_params_naive_bayes = randomized_search_naive_bayes.best_params_
print("Mejores Hiperparámetros para Naive Bayes:", best_params_naive_bayes)

Mejores Hiperparámetros para Naive Bayes: {'var_smoothing': 0.008111308307896872}


Ensamble de Modelos (Votación Mayoritaria)

La votación mayoritaria es una técnica de ensamble que combina las predicciones de varios modelos individuales y emite una predicción final basada en la mayoría de votos. En este caso, utilizaremos los modelos KNN, SVM y Naive Bayes para formar nuestro ensamble.
Inicialización de Modelos: Creamos instancias de los modelos KNN, SVM y Naive Bayes con los mejores hiperparámetros encontrados durante la optimización.
Configuración del Ensamble: Configuramos un clasificador de votación mayoritaria (VotingClassifier) que incluye los modelos KNN, SVM y Naive Bayes. Utilizamos el enfoque de 'hard' voting, donde la clase predicha es la clase mayoritaria votada por los clasificadores individuales.
Entrenamiento del Ensamble: Entrenamos el ensamble de modelos utilizando los datos escalados (X_scaled y y).
Predicciones y Evaluación: Hacemos predicciones en el conjunto de prueba y evaluamos el rendimiento del ensamble utilizando la matriz de confusión y el informe de clasificación.

In [14]:
# Inicializar los modelos con los mejores hiperparámetros encontrados
knn_classifier = KNeighborsClassifier(n_neighbors=best_params_knn['n_neighbors'],
                                      weights=best_params_knn['weights'],
                                      metric=best_params_knn['metric'])

svm_classifier = SVC(C=best_params_svm['C'],
                     kernel=best_params_svm['kernel'],
                     gamma=best_params_svm['gamma'],
                     random_state=42)

naive_bayes_classifier = GaussianNB(var_smoothing=best_params_naive_bayes['var_smoothing'])

# Configurar el ensamble de modelos usando Votación Mayoritaria
voting_classifier = VotingClassifier(estimators=[
    ('knn', knn_classifier),
    ('svm', svm_classifier),
    ('naive_bayes', naive_bayes_classifier)
], voting='hard')
# En 'hard' voting, la clase predicha es la clase mayoritaria votada por los clasificadores individuales.

# Entrenar el ensamble de modelos
voting_classifier.fit(X_scaled, y)

# Realizar predicciones en el conjunto de prueba
y_pred_ensemble = voting_classifier.predict(X_test)

# Evaluar el rendimiento del ensamble de modelos
conf_matrix_ensemble = confusion_matrix(y_test, y_pred_ensemble)
class_report_ensemble = classification_report(y_test, y_pred_ensemble)

# Imprimir la matriz de confusión y el informe de clasificación del ensamble de modelos
print("Matriz de Confusión (Ensamble - Votación Mayoritaria):")
print(conf_matrix_ensemble)
print("\nInforme de Clasificación (Ensamble - Votación Mayoritaria):")
print(class_report_ensemble)

Matriz de Confusión (Ensamble - Votación Mayoritaria):
[[187  19]
 [ 15 183]]

Informe de Clasificación (Ensamble - Votación Mayoritaria):
              precision    recall  f1-score   support

           0       0.93      0.91      0.92       206
           1       0.91      0.92      0.92       198

    accuracy                           0.92       404
   macro avg       0.92      0.92      0.92       404
weighted avg       0.92      0.92      0.92       404



Resumen del Proyecto:
Carga de Datos: Comenzamos importando el conjunto de datos desde un archivo CSV y exploramos su estructura para entender mejor las características disponibles.

Preprocesamiento de Datos: Eliminamos columnas innecesarias y convertimos las etiquetas de destino en valores numéricos para preparar los datos para el entrenamiento del modelo. También dividimos los datos en conjuntos de entrenamiento y prueba, y escalamos las características numéricas para algunos modelos.

Modelos Individuales: Entrenamos modelos de aprendizaje automático individuales, incluyendo KNN, SVM y Naive Bayes. Optimizamos los hiperparámetros de estos modelos utilizando técnicas como Grid Search y Randomized Search.

Ensamble de Modelos: Creamos un ensamble de modelos utilizando la técnica de Votación Mayoritaria. Este ensamble combina las predicciones de los modelos KNN, SVM y Naive Bayes para mejorar la precisión general del sistema.

Evaluación del Rendimiento: Utilizamos la matriz de confusión y el informe de clasificación para evaluar el rendimiento del ensamble de modelos. Estas métricas nos proporcionan una visión detallada de cómo el modelo clasifica las canciones en función de las preferencias del usuario.